In [2]:
""" Query wsdb for Gaia and GALEX magnitudes. """

##### Dependencies
import numpy as np
import pandas as pd
%matplotlib osx
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord as coord
import sqlutilpy
from astropy.io import fits

In [7]:
##### Load your wsdb.id
wsdbtmp = np.loadtxt('./wsdb.id',dtype=np.str_)
wsdbid = {'user':wsdbtmp[0],'password':wsdbtmp[1]}

In [8]:
##### Load in Kepler data
hdul = fits.open('./data/kepler2015.fits')
listcol = [hdul[1].data.__dict__['_coldefs'][i].name for i in range(len(hdul[1].data.__dict__['_coldefs']))]
acceptablewd = ['        WHITE DWARF D '+wdletter for wdletter in ['A','B','C','O','Q','X','Z']]
acceptrows = [i for i in range(len(hdul[1].data['CLASS'])) if hdul[1].data['CLASS'][i] in acceptablewd]
wdbox = {k:hdul[1].data[k][acceptrows] for k in listcol}
wd_x = [hdul[1].data[k][acceptrows] for k in listcol]

In [9]:
wdbox.keys()

dict_keys(['NAME', 'RA', 'DEC', 'GMAG', 'GMAG_ERROR', 'SPECT_TYPE', 'T_EFF', 'T_EFF_ERROR', 'LOG_G', 'PLATE_NUMBER', 'SPECTRUM_DATE', 'FIBER_NUMBER', 'LII', 'BII', 'GMAG_SNR', 'UMAG', 'UMAG_ERROR', 'RMAG', 'RMAG_ERROR', 'IMAG', 'IMAG_ERROR', 'ZMAG', 'ZMAG_ERROR', 'PM_TOT', 'LOG_G_ERROR', 'T_EFF_3D', 'T_EFF_3D_ERROR', 'LOG_G_3D', 'LOG_G_3D_ERROR', 'MASS', 'MASS_ERROR', 'COMMENTS', 'CLASS'])

In [10]:
for wd in np.unique(wdbox['CLASS']):
    print(wd, np.where(wdbox['CLASS']==wd)[0].shape[0])

        WHITE DWARF D A 6428
        WHITE DWARF D B 310
        WHITE DWARF D C 326
        WHITE DWARF D O 29
        WHITE DWARF D Q 71
        WHITE DWARF D X 2
        WHITE DWARF D Z 394


In [11]:
##### Query WSDB to get Gaia data 
wd_keys = ['wd'+lc for lc in listcol]
gaia_keys = ['source_id', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_weight_al', 'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error', 'mean_varpi_factor_al', 'astrometric_matched_observations', 'visibility_periods_used', 'astrometric_sigma5d_max', 'frame_rotator_object_type', 'matched_observations', 'duplicated_source', 'phot_g_n_obs', 'phot_g_mean_flux', 'phot_g_mean_flux_error', 'phot_g_mean_flux_over_error', 'phot_g_mean_mag', 'phot_bp_n_obs', 'phot_bp_mean_flux', 'phot_bp_mean_flux_error', 'phot_bp_mean_flux_over_error', 'phot_bp_mean_mag', 'phot_rp_n_obs', 'phot_rp_mean_flux', 'phot_rp_mean_flux_error', 'phot_rp_mean_flux_over_error', 'phot_rp_mean_mag', 'phot_bp_rp_excess_factor', 'phot_proc_mode', 'bp_rp', 'bp_g', 'g_rp', 'phot_variable_flag', 'l', 'b', 'a_g_val', 'e_bp_min_rp_val', 'radial_velocity', 'radial_velocity_error', 'rv_nb_transits','rv_template_teff','rv_template_logg','rv_template_fe_h','ebv']
galex_keys = ['galex_ra','galex_dec','fuv_mag','fuv_magerr','nuv_mag','nuv_magerr']
allwise_keys = ['allwise_ra','allwise_dec','w1mpro','w1sigmpro','w2mpro','w2sigmpro']

In [ ]:
### Cross-match with Gaia source
gaia_x = sqlutilpy.local_join("""
   select CAST(source_id as real), ra, ra_error, dec, dec_error, parallax, parallax_error, parallax_over_error, pmra, pmra_error, pmdec, pmdec_error, ra_dec_corr, ra_parallax_corr, ra_pmra_corr, ra_pmdec_corr, dec_parallax_corr, dec_pmra_corr, dec_pmdec_corr, parallax_pmra_corr, parallax_pmdec_corr, pmra_pmdec_corr, CAST(astrometric_n_obs_al as real), CAST(astrometric_n_obs_ac as real), CAST(astrometric_n_good_obs_al as real), CAST(astrometric_n_bad_obs_al as real), astrometric_gof_al, astrometric_chi2_al, astrometric_excess_noise, astrometric_excess_noise_sig, CAST(astrometric_params_solved as real), astrometric_primary_flag, astrometric_weight_al, astrometric_pseudo_colour, astrometric_pseudo_colour_error, mean_varpi_factor_al, CAST(astrometric_matched_observations as real), CAST(visibility_periods_used as real), astrometric_sigma5d_max, CAST(frame_rotator_object_type as real), CAST(matched_observations as real), duplicated_source, CAST(phot_g_n_obs as real), phot_g_mean_flux, phot_g_mean_flux_error, phot_g_mean_flux_over_error, phot_g_mean_mag, CAST(phot_bp_n_obs as real), phot_bp_mean_flux, phot_bp_mean_flux_error, phot_bp_mean_flux_over_error, phot_bp_mean_mag, CAST(phot_rp_n_obs as real), phot_rp_mean_flux, phot_rp_mean_flux_error, phot_rp_mean_flux_over_error, phot_rp_mean_mag, phot_bp_rp_excess_factor, CAST(phot_proc_mode as real), bp_rp, bp_g, g_rp, phot_variable_flag, l, b, a_g_val, e_bp_min_rp_val, radial_velocity, radial_velocity_error, CAST(rv_nb_transits as real),rv_template_teff,rv_template_logg,rv_template_fe_h,ebv from mytable as m left join lateral (select * from gaia_dr2.gaia_source as g where 
               q3c_join(m.wdra, m.wddec,g.ra,g.dec,1./3600)
       order by q3c_dist(m.wdra,m.wddec,g.ra,g.dec) asc limit 1)
       as tt on  true  order by xid""",'mytable',
       (wdbox['RA'],wdbox['DEC'],np.arange(len(wdbox['RA']))),('wdra','wddec','xid'),
           host='cappc127.ast.cam.ac.uk', user=wsdbid['user'], password=wsdbid['password'],
       preamb='set enable_seqscan to off; set enable_mergejoin to off; set enable_hashjoin to off;')
print("Finished Gaia cross-match, "+str(np.where(np.isnan(gaia_x[0])==False)[0].shape[0])+" sources.")                

### Cross-match with GALEX
galex_x = sqlutilpy.local_join("""
   select ra,dec,fuv_mag,fuv_magerr,nuv_mag,nuv_magerr from mytable as m left join lateral (select * from galex_bst17.main_fov055 as g where 
               q3c_join(m.wdra, m.wddec,g.ra,g.dec,10./3600)
       order by q3c_dist(m.wdra,m.wddec,g.ra,g.dec) asc limit 1)
       as tt on  true  order by xid""",'mytable',
       (wdbox['RA'],wdbox['DEC'],np.arange(len(wdbox['RA']))),('wdra','wddec','xid'),
           host='cappc127.ast.cam.ac.uk', user=wsdbid['user'], password=wsdbid['password'],
       preamb='set enable_seqscan to off; set enable_mergejoin to off; set enable_hashjoin to off;')
print("Finished GALEX cross-match, "+str(np.where(np.isnan(galex_x[0])==False)[0].shape[0])+" sources.")

### Cross-match with ALLWISE
allwise_x = sqlutilpy.local_join("""
   select ra, dec, w1mpro, w1sigmpro, w2mpro, w2sigmpro from mytable as m left join lateral (select * from allwise.main as g where 
               q3c_join(m.wdra, m.wddec,g.ra,g.dec,10./3600)
       order by q3c_dist(m.wdra,m.wddec,g.ra,g.dec) asc limit 1)
       as tt on  true  order by xid""",'mytable',
       (wdbox['RA'],wdbox['DEC'],np.arange(len(wdbox['RA']))),('wdra','wddec','xid'),
           host='cappc127.ast.cam.ac.uk', user=wsdbid['user'], password=wsdbid['password'],
       preamb='set enable_seqscan to off; set enable_mergejoin to off; set enable_hashjoin to off;')
print("Finished ALLWISE cross-match, "+str(np.where(np.isnan(allwise_x[0])==False)[0].shape[0])+" sources.") 

Finished Gaia cross-match, 4904 sources.
Finished GALEX cross-match, 4387 sources.


In [9]:
##### Merge together
x = wd_x+gaia_x+galex_x+allwise_x
keys = wd_keys+gaia_keys+galex_keys+allwise_keys
gagawdbox = {keys[i]:x[i] for i in range(len(x))}

In [10]:
##### Save cross-match
np.savez_compressed('./data/gagawd.npz',gagawdbox=gagawdbox)